# Analysis of Transitions
With our implemented Heuristic Process Miner we mined transitions in a process. Let's reverse engineer them and see if we can explain, what is happening on the Ethereum Blockchain

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
path_contracts_lookup = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/contractsWithERCFlags.csv'
path_block_times = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/blockTimes.csv'

In [3]:
ts = pd.read_csv('../../parity_transactions/rgd_1523059200_to_1524528000.csv', index_col=0)

In [4]:
ts

,CtC->CtC,CtC->CtU,CtC->UtU,CtC->end,CtU->CtC,CtU->CtU,CtU->UtC,CtU->UtU,CtU->end,UtC->CtC,UtC->CtU,UtC->UtC,UtC->UtU,UtC->end,UtU->CtC,UtU->CtU,UtU->UtC,UtU->UtU,UtU->end,sta->CtC,sta->CtU,sta->UtC,sta->UtU,total_events
day,,,,,,,,,,,,,,,,,,,,,,,,
1523059200000000000,3644,257,0,14,98,13,0,0,5,744,153,0,0,13,0,0,0,0,10,22,0.0,2443,2317,9.138355e+18
1523145600000000000,409560,40639,0,53574,30086,10735,0,0,30849,66147,21964,0,0,229435,0,0,0,0,259778,133,0.0,324546,268339,9.138874e+18
1523232000000000000,527713,32792,0,21920,18405,10861,0,0,9194,75365,17934,21,0,58682,0,0,0,0,79661,243,0.0,312377,295444,2.132525e+19
1523318400000000000,481590,45377,0,39946,35747,55599,0,0,17749,70366,18796,0,0,145202,0,0,0,0,194030,99,0.0,292383,276304,1.218655e+19
1523404800000000000,593556,56604,0,49423,46485,82875,0,0,19856,77733,18861,0,0,169408,0,0,0,0,185035,17,2.0,331382,276265,1.523405e+19
1523491200000000000,580596,30188,0,15991,17973,21309,0,0,6364,77943,15084,4,0,62977,0,0,0,0,82502,20,0.0,370173,319853,9.140947e+18
1523577600000000000,529568,25297,0,76059,14696,27605,0,0,27829,90966,17918,46,0,264359,0,0,0,0,343873,11,0.0,379062,349888,6.094310e+18
1523664000000000000,579405,23645,0,24610,14493,4293,0,0,6643,92045,13003,0,0,65303,0,0,0,0,66690,11,0.0,338424,271131,2.285496e+19
1523750400000000000,653266,28377,0,53692,16476,8500,0,0,19427,92682,15236,0,0,165027,0,0,0,0,200592,25,0.0,340373,278364,1.676125e+19


Breakdown of Transitions:
*  CtC->CtC: These events are event only invoking contract activities. They are resulting from a contract calling another contract or itself followed by another transaction of a contract calling another contract or itself
*  CtC->CtU: These events can be explained by traces, where a contract executes some logic involving a transaction to external owned user account. Many applications can be an example for that, like crowd funding like situations or any kind of contract representing a composite transaction structure with payouts to one or more externally owned accounts.
*  CtC->UtU: We are not having within one case a contract to contract transaction which is being directly followed by a user to user transaction, since user to X transactions can only be triggered from extern. However, we can see, that one day, this transition happens for times (-> **EXAMINE**)
*  CtC->end: these kind of transaction are common in the executing of any smart contract, which doesn't have a transaction to a user account at the end of it's execution.
*  CtU->CtC and CtU->CtU: these transitions make sense when we revisit how or event log is generated: the log is a linear sequential abstraction of the subtraces in the smart contract execution, which themselves have more complex tree strucutres. Any comisite of CtU..CtU tranansitions mean that there are several exernally owned accounts getting paid by a smart contract. If the composite includes a CtC transation this means that the logic also includes a intermediate step.
*  CtU->UtC and CtU->UtU: such transitions don't exist because user transactions can only get triggered from extern.
*  CtU->end: is a contract invoking a transaction to an externally owned account as last activity before its termination of execution.
*  UtC->CtC: this transition is the "starting" of the programming logic (CtC) in a smart contract by getting triggered by an externally owned account (UtC). This is the way of how contracts can get started.
*  UtC->CtU: "forwarding" events (for example ERC20 tokens). A user issues a transactionto a contract, which is directly afterwards issuing another transaction to another user.
*  UtC->UtC, UtC->UtU: UtC->UtC is not so easily explainable. If a user transfers something to a contract account, the contract account should be the next one to be able to issue a transaction in the same trace and not the user, since externally owned accounts do not have logic inside them. UtC->UtU has the same explaination and according to our reasoning it makes sense, that there is no occurence of that event. (-> **Examine UtC->UtC**)
*  UtC->end: this is a simple transaction where a user transfers an amount to a contract and the contract "stores" it, meaning does no more computational steps afterwards.
*  UtU->CtC, UtU->CtU, UtU->UtC, UtU->UtU: all these transition to occurr 0 times in our sample, which is explainable that after a user issued a transaction to another external user, everything what the second one would do with the amount, would be in another transaction / case.
*  UtU->end: these are simple (BTC like transaction) where a user sends an amount of ether to another user and then does nothing else with it.
*  sta->CtC, sta->CtU: these seem to be contract transactions being triggered "out of thin air" (meanining not by an externally owned user account). **Examine**
*  sta->UtU, sta->UtC: regular starting events. Nothing special to observe.

From our transition explaination we are now examining the points, which we cannot understand easily to find out what is happening there.

## CtC->UtU - Why is this happening?
In our sample time frame this exactly happens once, at timestamp 1524096000000000000, translation to April 19th 2018.

In [5]:
transitions_19_1 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/mr_2018-04-16 00:00:00_in_transactions5450000-5459999_4_transitions.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
import glob, os
os.chdir("/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/")
for file in glob.glob("*.csv"):
    print(file)

transactions5440000-5449999.csv
mr_2018-04-16 00:00:00_in_transactions5440000-5449999_0_transitions.csv
transactions5500000-5500000.csv
rgd_1523232000_to_1524528000.csv
mr_2018-04-19 00:00:00_in_transactions5460000-5469999_8_confidence.csv
transactions5490000-5499999.csv
mr_2018-04-16 00:00:00_in_transactions5450000-5459999_4_confidence.csv
transactions5410000-5419999.csv
mr_2018-04-08 00:00:00_in_transactions5400000-5409999_5_transitions.csv
rgd_1523664000_to_1524528000.csv
mr_2018-04-22 00:00:00_in_transactions5480000-5489999_6_transitions_agg.csv
mr_2018-04-23 00:00:00_in_transactions5480000-5489999_6_transitions.csv
ps_transactions5400000-5409999_trace_lengths.csv
ps_transactions5460000-5469999_trace_lengths.csv
mr_2018-04-14 00:00:00_in_transactions5430000-5439999_7_confidence.csv
ps_transactions5450000-5459999_event_log.csv
mr_2018-04-23 00:00:00_in_transactions5490000-5499999_2_transitions_agg.csv
rgtl_1523059200_to_1524528000.csv
mr_2018-04-13 00:00:00_in_transactions5430000-54

In [7]:
transitions_19_1.groupby('transition').count()

,total_pos,timestamp
transition,,
CtC->CtC,372713,372713
CtC->CtU,18419,18419
CtC->end,53966,53966
CtU->CtC,9638,9638
CtU->CtU,3312,3312
CtU->end,17084,17084
UtC->CtC,64281,64281
UtC->CtU,8651,8651
UtC->UtC,1,1


As we can see the 4 events do not occure in that file. We have occassionally more then one csv file per saved day since our raw input files are grouped by blocks and not day (-> overlap)

In [8]:
transitions_19_2 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/mr_2018-04-19 00:00:00_in_transactions5460000-5469999_8_transitions.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
transitions_19_2.groupby('transition').count()

,total_pos,timestamp
transition,,
CtC->CtC,424077,424077
CtC->CtU,15326,15326
CtC->UtU,4,4
CtU->CtC,6879,6879
CtU->CtU,9701,9701
UtC->CtC,66308,66308
UtC->CtU,10363,10363
sta->CtC,773,773
sta->UtC,387439,387439


There our not explainable CtC->UtU transactions are.

In [10]:
ccuus = transitions_19_2[transitions_19_2['transition']=='CtC->UtU']

In [11]:
ccuus

,total_pos,timestamp,transition
4015020,5467829002004747,1524132332,CtC->UtU
4074762,5467943002034763,1524134036,CtC->UtU
4086084,5467969002040454,1524134430,CtC->UtU
4118586,5468031002056786,1524135398,CtC->UtU


In [12]:
timestamp = 1524132332

In [13]:
block_times = pd.read_csv(path_block_times, index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
transaction_lookup = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/transaction_lookup.csv')

In [15]:
block_times[block_times['timestamp']==1524132332]

,timestamp
number,
5467829,1524132332


In [16]:
log_546 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/ps_transactions5460000-5469999_event_log.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
candidate_tx_hashes = log_546.merge(ccuus, on='total_pos').merge(transaction_lookup, left_on='transaction_id', right_on='id')['transaction_hash']
candidate_tx_hash_list = candidate_tx_hashes.values.tolist()

In [18]:
raw_545 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/transactions5460000-5469999.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
raw_545[raw_545['transactionHash'].isin(candidate_tx_hash_list)]

,action.address,action.balance,action.callType,action.from,action.gas,action.init,action.input,action.refundAddress,action.to,action.value,blockHash,blockNumber,error,result,result.address,result.code,result.gasUsed,result.output,subtraces,traceAddress,transactionHash,transactionPosition,type
2004741,NaN,NaN,call,0x62505a9cc4b75d2a454523a05cf11186774bf222,0xef038,NaN,0x,NaN,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,NaN,NaN,0x41d61,0x,3,[],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call
2004742,NaN,NaN,NaN,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0xe38c3,0x608060405234801561001057600080fd5b5061015780...,NaN,NaN,NaN,0x0,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0x608060405260043610610041576000357c0100000000...,0x10c6d,NaN,0,[0],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,create
2004743,NaN,NaN,NaN,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0xcb4f4,0x608060405234801561001057600080fd5b5061013980...,NaN,NaN,NaN,0x0,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,0x13ea5e193d651abb4eae5ff09694ef784e6ec405,0x608060405260043610610041576000357c0100000000...,0xf4f7,NaN,0,[1],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,create
2004744,NaN,NaN,call,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0x18f9c,NaN,0xddb4965200000000000000000000000013ea5e193d65...,NaN,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,NaN,NaN,0x102c8,0x,1,[2],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call
2004745,NaN,NaN,delegatecall,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0x18561,NaN,0x15d55b28,NaN,0x13ea5e193d651abb4eae5ff09694ef784e6ec405,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,Reverted,NaN,NaN,NaN,NaN,NaN,1,"[2, 0]",0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call
2004746,NaN,NaN,NaN,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0x10377,0x60806040526000733115a65bd094867f00e75b6173e9...,NaN,NaN,NaN,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,Reverted,NaN,NaN,NaN,NaN,NaN,1,"[2, 0, 0]",0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,create
2004747,NaN,NaN,call,0x806b26879bc0e7f1de43399073a6ebaee818d0e7,0x8fc,NaN,0x,NaN,0x3115a65bd094867f00e75b6173e93dcd921bdda5,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,NaN,NaN,0x0,0x,0,"[2, 0, 0, 0]",0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call
2034757,NaN,NaN,call,0x62505a9cc4b75d2a454523a05cf11186774bf222,0xef038,NaN,0x,NaN,0xb3a1a216e156c37a34c2c2c68754196f6bb44ac5,0xe8d4a51000,0xdd351a13b2eb723cf5a7002cc3c4fbbe854b01036a75...,5467943,NaN,NaN,NaN,NaN,0x4a41d,0x,3,[],0x5b6b4bb20e0b0f71377fe942bff7688eb2be6878915b...,67.0,call
2034758,NaN,NaN,NaN,0xb3a1a216e156c37a34c2c2c68754196f6bb44ac5,0xe38c3,0x608060405234801561001057600080fd5b5061015780...,NaN,NaN,NaN,0x0,0xdd351a13b2eb723cf5a7002cc3c4fbbe854b01036a75...,5467943,NaN,NaN,0x68cb39d6b508090e1b274281a343ae173b138e8f,0x608060405260043610610041576000357c0100000000...,0x10c6d,NaN,0,[0],0x5b6b4bb20e0b0f71377fe942bff7688eb2be6878915b...,67.0,create
2034759,NaN,NaN,NaN,0xb3a1a216e156c37a34c2c2c68754196f6bb44ac5,0xcb4f4,0x608060405234801561001057600080fd5b5061013980...,NaN,NaN,NaN,0x0,0xdd351a13b2eb723cf5a7002cc3c4fbbe854b01036a75...,5467943,NaN,NaN,0xe125a3764942be5ea77bba6668d56e71036db3c9,0x608060405260043610610041576000357c0100000000...,0xf4f7,NaN,0,[1],0x5b6b4bb20e0b0f71377fe942bff7688eb2be6878915b...,67.0,create


In [20]:
raw_545[raw_545['transactionHash'] == candidate_tx_hash_list[0]]

,action.address,action.balance,action.callType,action.from,action.gas,action.init,action.input,action.refundAddress,action.to,action.value,blockHash,blockNumber,error,result,result.address,result.code,result.gasUsed,result.output,subtraces,traceAddress,transactionHash,transactionPosition,type
2004741,NaN,NaN,call,0x62505a9cc4b75d2a454523a05cf11186774bf222,0xef038,NaN,0x,NaN,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,NaN,NaN,0x41d61,0x,3,[],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call
2004742,NaN,NaN,NaN,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0xe38c3,0x608060405234801561001057600080fd5b5061015780...,NaN,NaN,NaN,0x0,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0x608060405260043610610041576000357c0100000000...,0x10c6d,NaN,0,[0],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,create
2004743,NaN,NaN,NaN,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0xcb4f4,0x608060405234801561001057600080fd5b5061013980...,NaN,NaN,NaN,0x0,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,0x13ea5e193d651abb4eae5ff09694ef784e6ec405,0x608060405260043610610041576000357c0100000000...,0xf4f7,NaN,0,[1],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,create
2004744,NaN,NaN,call,0xd646ad69bf3f3498d243b44257a1b559c5b82a0d,0x18f9c,NaN,0xddb4965200000000000000000000000013ea5e193d65...,NaN,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,NaN,NaN,0x102c8,0x,1,[2],0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call
2004745,NaN,NaN,delegatecall,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0x18561,NaN,0x15d55b28,NaN,0x13ea5e193d651abb4eae5ff09694ef784e6ec405,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,Reverted,NaN,NaN,NaN,NaN,NaN,1,"[2, 0]",0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call
2004746,NaN,NaN,NaN,0x346abe31c408f57e4e2064ead21245fb1d4ec7b2,0x10377,0x60806040526000733115a65bd094867f00e75b6173e9...,NaN,NaN,NaN,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,Reverted,NaN,NaN,NaN,NaN,NaN,1,"[2, 0, 0]",0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,create
2004747,NaN,NaN,call,0x806b26879bc0e7f1de43399073a6ebaee818d0e7,0x8fc,NaN,0x,NaN,0x3115a65bd094867f00e75b6173e93dcd921bdda5,0xe8d4a51000,0x751232bda16cc850d1d0836c8961758dac507599604c...,5467829,NaN,NaN,NaN,NaN,0x0,0x,0,"[2, 0, 0, 0]",0x90f43e6e82b0305fcb6addf312d33a4c51f20b6f8af7...,105.0,call


We can see, in all instances such an unexpected CtC->UtU transaction occures, reverted contract creation operations occurr. Since we are in our initial data cleaning step we get rid of all subtraces which don't have type == call the reversion of the UtU transaction gets shifted and gets represented as a CtC->UtU transaction.

## UtC->UtC - Why is this happening?
We found 46 occurence at day 1523577600000000000 of UtC->UtC. We will examine that next

In [21]:
transitions_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/mr_2018-04-13 00:00:00_in_transactions5430000-5439999_7_transitions.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [22]:
ucucs = transitions_543[transitions_543['transition']=='UtC->UtC']

In [23]:
ucucs.head()

,total_pos,timestamp,transition
1108776,5432219000545019,1523608857,UtC->UtC
1108778,5432219000545020,1523608857,UtC->UtC
1122114,5432248000551726,1523609218,UtC->UtC
1122116,5432248000551727,1523609218,UtC->UtC
1135380,5432273000558389,1523609537,UtC->UtC


From only observing the total position tags we can see, that these are coming in pairs, which means that we have a sequence of 3 UtC events directly successing each other. So three UtC in a row?

In [24]:
block_times = pd.read_csv(path_block_times, index_col=0)
transaction_lookup = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/transaction_lookup.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [25]:
candidate_tx_hashes_ucuc = log_546.merge(ucucs, on='total_pos').merge(transaction_lookup, left_on='transaction_id', right_on='id')['transaction_hash']
candidate_tx_hash_ucuc_list = candidate_tx_hashes_ucuc.values.tolist()

In [26]:
raw_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/transactions5430000-5439999.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
contracts_lookup = pd.read_csv(path_contracts_lookup, index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [28]:
res_543 = raw_543[raw_543['transactionHash'].isin(candidate_tx_hash_list)].merge(contracts_lookup, left_on='action.from', right_on='result.address', how='left').merge(contracts_lookup, how='left', left_on='action.to', right_on='result.address')
res_543

,action.address,action.balance,action.callType,action.from,action.gas,action.init,action.input,action.refundAddress,action.to,action.value,blockHash,blockNumber,error,result,result.address_x,result.code,result.gasUsed,result.output,subtraces,traceAddress,transactionHash,transactionPosition,type,result.address_y,isERC20_x,timestamp_x,result.address,isERC20_y,timestamp_y


In [29]:
transactions_parsed_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/ps_transactions5430000-5439999_event_log.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [31]:
raw_543

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


What we can see here are create statement which themselves have sub create statments (contract factories)

In [31]:
contracts_lookup.head()

,result.address,isERC20,timestamp
blockNumber,,,
100490,0x3c7771db7f343b79003e8c9ba787bbe47764ed05,False,1439807307
100885,0x449c5b639e9852ada644ffaacfe325dfce6e6e0a,False,1439813909
101285,0x6e146dc6fd2c5a7308dbe65a57e38df154db7b28,False,1439820467
101358,0x6b211fdc0d944a84bba8534d631fad9262ba80b8,False,1439821781
101535,0x8efe9a62099820c58f81bb1573d8619c7f9dffbe,False,1439824347
